# Analysis of Projects GPT V2

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

/Users/altaf/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

store = Chroma(persist_directory="./chroma_db", embedding_function=embeddings, collection_name='projects' )

In [5]:
store.similarity_search("Which projects eleviate poverty?")

[Document(page_content='Support vulnerable households in rural communities to have improved food security, household income, and resilience to climate change through the construction of agro-based infrastructure projects in the provinces of Badakhshan, Faryab, and Ghazni.', metadata={'source': 'AFG1037'}),
 Document(page_content='The overall goal of this pilot intervention is to support vulnerable livelihoods and reduce food insecurity in vulnerable communities in southern Afghanistan.\nThe project is also expected to support the reduction of illicit crops cultivation, while, at the same time, promoting the economic empowerment of women.', metadata={'source': 'AFG1027'}),
 Document(page_content='Improve food security of vulnerable households through diversified Livelihood opportunities and Increase Agriculture and Livestock Production as main sources of livelihood for targeted communities.', metadata={'source': 'AFG1032'}),
 Document(page_content=' Project Development Objective is as f

In [6]:
from langchain.chat_models import ChatOpenAI

retriever = store.as_retriever()

#retriever = store.as_retriever(search_type="mmr", search_kwargs={'k': 6, 'lambda_mult': 0.25})

llm = ChatOpenAI(temperature = 0.0, model_name="gpt-3.5-turbo-16k-0613")

In [7]:
from langchain.prompts import PromptTemplate
prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Return a clear answer and a one sentence summary"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [8]:
from langchain.chains import RetrievalQA

#chain_type="stuff", 

chain_type_kwargs = {"prompt": PROMPT}
qa = RetrievalQA.from_chain_type(llm=llm,                                 
                                 chain_type="stuff",
                                 retriever=retriever, 
                                 chain_type_kwargs=chain_type_kwargs,
                                 verbose=True)

In [9]:
from IPython.display import display, Markdown

query = "Provide a short summary on the energy projects"
res = qa.run(query)

display(Markdown(res.replace('.' ,'.\n\n')))



> Entering new RetrievalQA chain...

> Finished chain.


The energy projects mentioned in the context include the expansion and strengthening of electricity transmission networks, the transfer of electrical energy from Al-Dur station to load centers in Bahrain, and the contribution to the GCC electrical interconnection project.

 These projects aim to meet the increasing demand for electricity, alleviate overloaded subsystems, and improve energy trading capabilities within the GCC.

 

Summary: The energy projects aim to enhance electricity transmission, alleviate overloaded subsystems, and improve energy trading within the GCC.

